In [1]:
import sys
import os 
%load_ext autoreload
%autoreload 2

os.chdir("/home/cooper-cooper/Desktop/vans/")

sys.path[0] = "/home/cooper-cooper/Desktop/vans/"


from utilities.variational import Autoencoder
import tensorflow_quantum as tfq
from utilities.qmodels import QNN,EnergyLoss

import tensorflow as tf

from utilities.evaluator import Evaluator
import numpy as np

import matplotlib.pyplot as plt
from IPython import display
from tqdm import tqdm
from utilities.variational import VQE
import cirq

from datetime import datetime

from utilities.variational import VQE
from utilities.evaluator import Evaluator
from utilities.idinserter import IdInserter
from utilities.simplifier import Simplifier
from utilities.unitary_killer_autoencoder import UnitaryMurder
from utilities.misc import scheduler_selector_temperature, scheduler_parameter_perturbation_wall #this outputs always 10 for now.



### load pure states

indexed_pure_states=[]
resolver_pure_states=[]
js=np.linspace(0.5,2.3,16)[0::3]
for bond in js:
    problem_config ={"problem" : "H2", "geometry": [('H', (0., 0., 0.)), ('H', (0., 0., bond))], "multiplicity":1, "charge":0, "basis":"sto-3g"}
    args={"n_qubits":8,"problem_config":problem_config, "load_displaying":False,"specific_folder_name":"4_bd_{}".format(bond)}
    evaluator = Evaluator(args,loading=True, path="../data-vans/data-in-paper-2marzo/")
    #energies.append(evaluator.raw_history[len(list(evaluator.raw_history.keys()))-1][-1])
    vqe_handler = VQE(n_qubits=4,noise_config={}, problem_config=problem_config,
                            return_lower_bound=True)
    _, cuener, indexed_circuit, resolver, bestener = evaluator.evolution[evaluator.get_best_iteration()]
    indexed_pure_states.append(indexed_circuit)
    resolver_pure_states.append(resolver)

In [ ]:

auto_handler = Autoencoder(n_qubits=4,problem_config=problem_config, nb=2, lr = 1.,
                        epochs=2000, verbose=0,
                        patience=200, optimizer="adam",
                        many_indexed_circuits=indexed_pure_states, many_symbols_to_values=resolver_pure_states)

evaluator = Evaluator(args={"n_qubits":4,
                            "problem_config":{"problem":"AutoencoderH2","n_qubits":3},
                            "noise_config":{}},
                      info="", 
                      path="../data-vans/", acceptance_percentage=0.01, 
                      accuracy_to_end=0., reduce_acceptance_percentage=True)

info = "AUTOENCODEr\ntotal_qubits: {}\ntrash_qubits: {}\n".format(len(auto_handler.qubits), auto_handler.nb)
evaluator.displaying["information"]+=info


start = datetime.now()

initialization="hea"

selector_temperature=20
reps=50
rate_iids_per_step=1.0
wait_to_get_back=10

iid = IdInserter(n_qubits=len(auto_handler.qubits))
iid.selector_temperature = selector_temperature

#Simplifier reduces gates number as much as possible while keeping same expected value of target hamiltonian
Simp = Simplifier(n_qubits=len(auto_handler.qubits))

#UnitaryMuerder is in charge of evaluating changes on the energy while setting apart one (or more) parametrized gates. If
killer = UnitaryMurder(auto_handler, many_indexed_circuits=indexed_pure_states, many_symbols_to_values=resolver_pure_states, noise_config={})


indexed_circuit = auto_handler.hea_ansatz_indexed_circuit(L=1)

print("beggining to train!")
energy, symbol_to_value, training_evolution = auto_handler.autoencoder(indexed_circuit) #compute energy
#add initial info to evaluator


to_print="\nIteration #{}\nTime since beggining:{}\n best energy: {}\ncurrent_energy {}\n lower_bound: {}".format(0, datetime.now()-start, energy, energy, evaluator.accuracy_to_end)
print(to_print)
evaluator.displaying["information"]+=to_print

evaluator.add_step(indexed_circuit, symbol_to_value, energy, relevant=True)
evaluator.lowest_energy = energy

for iteration in range(1,reps+1):
    relevant=False

    ### create a mutation M (maybe this word is too fancy); we add (probably more than one) identity resolution
    iid.selector_temperature=scheduler_selector_temperature(energy, evaluator.lowest_energy, when_on=selector_temperature)
    M_indices, M_symbols_to_values, M_idx_to_symbols = iid.place_identities(indexed_circuit, symbol_to_value, rate_iids_per_step= rate_iids_per_step)

    ### simplify the circuit as much as possible
    Sindices, Ssymbols_to_values, Sindex_to_symbols = Simp.reduce_circuit(M_indices, M_symbols_to_values, M_idx_to_symbols)

    ## compute the energy of the mutated-simplified circuit [Note 1]
    MSenergy, MSsymbols_to_values, _ = auto_handler.autoencoder(
        Sindices, symbols_to_values=Ssymbols_to_values,
        parameter_perturbation_wall=scheduler_parameter_perturbation_wall(its_without_improvig=evaluator.its_without_improvig))

    if evaluator.accept_energy(MSenergy):
        indexed_circuit, symbol_to_value, index_to_symbols = Sindices, MSsymbols_to_values, Sindex_to_symbols
        # unitary slaughter: delete as many 1-qubit gates as possible, as long as the energy doesn't go up (we allow %1 increments per iteration)
        cnt=0
        reduced=True
        lmax=len(indexed_circuit)
        while reduced and cnt < lmax:
            indexed_circuit, symbol_to_value, index_to_symbols, energy, reduced = killer.unitary_slaughter(indexed_circuit, symbol_to_value, index_to_symbols, reference_energy=MSenergy)
            indexed_circuit, symbol_to_value, index_to_symbols = Simp.reduce_circuit(indexed_circuit, symbol_to_value, index_to_symbols)
            cnt+=1
        relevant=True
    evaluator.add_step(indexed_circuit, symbol_to_value, energy, relevant=relevant)

    to_print="\nIteration #{}\nTime since beggining:{}\n best energy: {}\ncurrent energy: {}\n lower_bound: {}".format(iteration, datetime.now()-start, evaluator.lowest_energy,energy, evaluator.accuracy_to_end)
    print(to_print)
    evaluator.displaying["information"]+=to_print

    evaluator.save_dicts_and_displaying()

    if evaluator.if_finish_ok is True:
        print("HOMEWORK DONE! \nBeers on me ;)")
        break

    if evaluator.its_without_improvig == wait_to_get_back:
        print("Getting back to favorite, it's been already {} iterations".format(wait_to_get_back))
        _, energy, indices, resolver, _, _ =  evaluator.evolution[evaluator.get_best_iteration()]
        evaluator.its_without_improvig = 0


beggining to train!

Iteration #0
Time since beggining:0:00:03.544222
 best energy: 0.07307714223861694
current_energy 0.07307714223861694
 lower_bound: 0.0
I killed 1 unitaries, Ef - Ei: -0.0004031062126159668
I killed 2 unitaries, Ef - Ei: -0.0004176497459411621
I killed 3 unitaries, Ef - Ei: -0.0004177093505859375
I killed 4 unitaries, Ef - Ei: -0.0004177093505859375
I killed 5 unitaries, Ef - Ei: -0.0004177093505859375
I killed 6 unitaries, Ef - Ei: -0.0004176497459411621
I killed 7 unitaries, Ef - Ei: -0.00038129091262817383
I killed 8 unitaries, Ef - Ei: -0.00038188695907592773
I killed 9 unitaries, Ef - Ei: -0.00038188695907592773

Iteration #1
Time since beggining:0:00:23.599728
 best energy: 0.07307714223861694
current energy: 0.07321840524673462
 lower_bound: 0.0
I killed 1 unitaries, Ef - Ei: 5.960464477539063e-08

Iteration #2
Time since beggining:0:00:26.712880
 best energy: 0.07307714223861694
current energy: 0.07307720184326172
 lower_bound: 0.0
I killed 1 unitaries, Ef 

In [ ]:
np.squeeze(MSenergy)<1

Now we construct the inverse circuit.

Let's say that VQE-VAns prepares the state $| \psi_i >$ as $V_i |0>$. Then VAns finds an autoencoder circuit $W$.

Then we'll check at the average fidelity $\sum_i |<\psi_i| \tilde{\psi}_i> |^{2}$

with  $|\tilde{\psi}_i> = (\mathcal{E} \circ \mathcal{M} \circ \mathcal{D}) \; |\psi_i>$; 

with $\mathcal{E} = W$, $\mathcal{D} = W^{\dagger}$ and $\mathcal{M} = \mathcal{I}_{AB} - \dfrac{1}{n_B} ( \sum_j |0>_{j_B}<0| \times \mathcal{I}_{A, \bar{j}})$

In [ ]:
W, infidelity, indexed_circuit, resolver, best_infidelity = evaluator.evolution[evaluator.get_best_iteration()]

In [ ]:
W = cirq.unitary(W)
Wdagger = cirq.unitary(auto_handler.give_inverse(indexed_circuit,resolver)[-1])

In [ ]:
def compute_fidelity(input_state, encoding, channel, decoding):
    """
    we assume input_state is a ket.
    """
    st = np.dot(encoding, input_state)
    st = np.dot(channel, st)
    st = np.dot(decoding, st)
    return np.round(np.dot(np.conjugate(input_state),st), 16)

In [ ]:
fidelities = []
for k in range(len(auto_handler.qbatch)):
    input_state = cirq.unitary(auto_handler.qbatch[k])[:,0]
    channel = np.eye(2**len(auto_handler.qubits)) - (1/auto_handler.nb)*(np.kron(np.kron(*[auto_handler.zero_proj(vqe_handler.qubits[k]).matrix() for k in range(2)]), np.eye(4)))
    fidelities.append(compute_fidelity(input_state, W, channel, Wdagger))

In [ ]:
np.log10(1-np.mean(fidelities))

Now we test on different bond lengths

In [ ]:
bond_train=np.linspace(0.5,2.3,16)[0::3]
bond_test=[]
test_energies = []
test_states=[]
train_energies=[]
fcis=[]
fcis_test=[]
fcis_train=[]
train_states=[]
for b in tqdm(np.linspace(0.5,2.3,16)):
    problem_config ={"problem" : "H2", "geometry": [('H', (0., 0., 0.)), ('H', (0., 0., b))], "multiplicity":1, "charge":0, "basis":"sto-3g"}
    args={"n_qubits":8,"problem_config":problem_config, "load_displaying":False,"specific_folder_name":"4_bd_{}".format(b)}
    evaluator = Evaluator(args,loading=True, path="../data-vans/")
    #energies.append(evaluator.raw_history[len(list(evaluator.raw_history.keys()))-1][-1])
    vqe_handler = VQE(n_qubits=4,noise_config={}, problem_config=problem_config,
                            return_lower_bound=True)
    VAnsCircuit, cuener, indexed_circuit, resolver, bestener = evaluator.evolution[evaluator.get_best_iteration()]
    
    model = QNN(symbols=list(resolver.keys()), observable=vqe_handler.observable, batch_sizes=1)
    model.compile( loss=EnergyLoss())
    fcis.append(vqe_handler.lower_bound_energy)
    
    if not (b in bond_train):
        bond_test.append(b)
        test_energies.append(np.squeeze(model(tfq.convert_to_tensor([VAnsCircuit]))))
        fcis_test.append(vqe_handler.lower_bound_energy)
        test_states.append(cirq.unitary(VAnsCircuit)[:,0])
        
    else:
        train_states.append(cirq.unitary(VAnsCircuit)[:,0])
        
        train_energies.append(np.squeeze(model(tfq.convert_to_tensor([VAnsCircuit]))))
        fcis_train.append(vqe_handler.lower_bound_energy)

In [ ]:
fcisfull=[]
for b in tqdm(np.linspace(0.5,2.3,100)):
    problem_config ={"problem" : "H2", "geometry": [('H', (0., 0., 0.)), ('H', (0., 0., b))], "multiplicity":1, "charge":0, "basis":"sto-3g"}
    args={"n_qubits":8,"problem_config":problem_config, "load_displaying":False,"specific_folder_name":"4_bd_{}".format(b)}
    vqe_handler = VQE(n_qubits=4,noise_config={}, problem_config=problem_config,
                            return_lower_bound=True)
    fcisfull.append(vqe_handler.lower_bound_energy)


In [ ]:
plt.figure(figsize=(10,10))
plt.plot(np.linspace(0.5,2.3,100),fcisfull,color="red",linewidth=5,alpha=0.5,label="FCI")
plt.scatter(bond_train, fcis_train,s=300, color="green",label="Train points")
plt.scatter(bond_test, fcis_test, s=300,color="blue",label="Test points")
plt.xlabel("bond length",size=20)
plt.ylabel("bond length",size=20)
plt.legend(prop={"size":20})
plt.savefig(fol+"energies.png")

In [ ]:
channel = np.eye(2**len(auto_handler.qubits)) - (1/auto_handler.nb)*(np.kron(np.kron(*[auto_handler.zero_proj(vqe_handler.qubits[k]).matrix() for k in range(2)]), np.eye(4)))

test_fidelities = []
for state in test_states:
    test_fidelities.append(compute_fidelity(state, W, channel, Wdagger))
    
train_fidelities = []
for state in train_states:
    train_fidelities.append(compute_fidelity(state, W, channel, Wdagger))

In [ ]:
import os 
fol="results/autoencoder/H2/"
os.makedirs(fol,exist_ok=True)
sts=["fcis_train", "fcis_test", "bond_train","bond_test", "bonds", "test_fidelities", "train_fidelities","fcisfull"]
fis=[fcis_train, fcis_test, bond_train, bond_test,np.linspace(0.5,2.3,16), test_fidelities, train_fidelities,fcisfull]
for strfi, thi in zip(sts, fis):
    np.save(fol+strfi,thi)

In [ ]:
bonds=np.linspace(0.5,2.3,16)
plt.figure(figsize=(10,10))
plt.plot(bonds,np.ones(len(bonds)),color="black",alpha=0.5,label="1")
plt.scatter(bond_train, train_fidelities, color="red",label="Train points")
plt.scatter(bond_test, test_fidelities, color="green",label="Test points")
plt.xlabel("bond length",size=20)
plt.ylabel("bond length",size=20)
plt.legend(prop={"size":20})
plt.savefig("fidelities.png")